In [2]:
import os
import numpy as np

import LMCE.cfusdlog as cfusdlog
import LMCE.uav_trajectory as uav_trajectory
from LMCE.error_estimation import error_calculator, find_best_cutoff

Here we compare the different average errors for each flight. The flights have a timescale of 0.6.

In [5]:
# Get desired path
traj = uav_trajectory.Trajectory()
traj.loadcsv("./LMCE/flight_paths/figure8.csv")
traj.stretchtime(2)

ts = np.arange(0, traj.duration, 0.01)
evals = np.empty((len(ts), 15))
for t, i in zip(ts, range(0, len(ts))):
    e = traj.eval(t)
    e.pos += np.array([0, 0, 1])
    evals[i, 0:3] = e.pos

target_pos = evals.transpose()

main_path = "./final_results/cf21/"
categories = ["lee", "nn", "standard", "tree"]
categories = ["nn"]


for c in categories:
    avg_errors = []
    avg_it = []
    path = main_path + c
    for filename in os.listdir(path):
        # if c == "nn" and filename == "nn_log05":
        #     continue

        data_path = path + "/" + filename
        # Get real path
        data = cfusdlog.decode(data_path)['fixedFrequency']
        real_pos = [data["stateEstimate.x"], data["stateEstimate.y"]]

        # Calculate error
        cutoff = find_best_cutoff(real_pos, target_pos)
        error = error_calculator(cutoff, real_pos, target_pos, vis=False)
        avg_errors.append(error)
        avg_it.append(sum(data["nn_perf.inf_tim"])/len(data["nn_perf.inf_tim"]))

    avg_error = sum(avg_errors)/len(avg_errors)
    avg_it = sum(avg_it)/len(avg_it)
    print("----------------------------------------")
    print(f"Avg. error for '{c}': {avg_error:.5f} (m) for the invidual flights: {avg_errors}")
    print(f"Avg. inf time for '{c}': {avg_it:.5f} (Hz)")


[3.67647052 3.67647052 3.67647052 ... 3.47222233 3.67647052 3.67647052]
[3.70370364 3.70370364 3.64963508 ... 3.67647052 3.73134327 3.73134327]
[3.57142854 3.57142854 3.52112675 ... 3.73134327 3.73134327 3.67647052]
[3.78787875 3.78787875 3.47222233 ... 3.67647052 3.67647052 3.67647052]
[3.57142854 3.67647052 3.57142854 ... 3.78787875 3.67647052 3.67647052]
[3.78787875 3.67647052 3.57142854 ... 3.67647052 3.67647052 3.78787875]
